In [ ]:
# Download the saved model from Google Storage
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip

In [ ]:
# Unzip the SavedModel downloaded from Google Stroage
!mkdir downloaded_gs_model # create new dir to store downloaded feature extraction model
!unzip 07_efficientnetb0_feature_extract_model_mixed_precision.zip -d downloaded_gs_model

In [ ]:
# Load and evaluate downloaded GS model
downloaded_gs_model = tf.keras.models.load_model("downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision")

In [ ]:
# Get a summary of the downloaded model
downloaded_gs_model.summary()

In [ ]:
# How does the loaded model perform? (evaluate it on the test dataset)
perform_downloaded_gs_model = downloaded_gs_model.evaluate(test_data)
perform_downloaded_gs_model

In [ ]:
# Set all of the layers .trainable variable in the loaded model to True (so they're unfrozen)
for layer in downloaded_gs_model.layers:
  print(layer.name, layer.trainable)

In [ ]:
# Unfreeze all of the layer in the base model
downloaded_gs_model.trainable = True

In [ ]:
for layer in downloaded_gs_model.layers:
  print(layer.name, layer.trainable)

In [ ]:
# Check to see what dtype_policy of the layers in your loaded model are
for layer in downloaded_gs_model.layers: # check the layers of the base model (layer at index 1 of `model`)
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
# Monitor the val_loss and stop training if it doesn't improve for 3 epochs
# See: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping for more
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=3)


# Create ModelCheckpoint callback to save best model during fine-tuning
# Save the best model only
# Monitor val_loss while training and save the best model (lowest val_loss)
# See: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint for more
checkpoint_filepath = "model_checkpoints2/cp.ckpt"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               monitor='val_loss',
                                                               save_best_only=True)

In [ ]:
# Creating learning rate reduction callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down
                                                 min_lr=1e-7)

In [ ]:
# Compile the model ready for fine-tuning
# Use the Adam optimizer with a 10x lower than default learning rate
downloaded_gs_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # learning rate lower by 10x
              metrics=["accuracy"])


In [ ]:
# Start to fine-tune (all layers)
# Use 100 epochs as the default
# Validate on 15% of the test_data
# Use the create_tensorboard_callback, ModelCheckpoint and EarlyStopping callbacks you created eaelier
# YOUR_CODE_HERE
history_101_food_classes_feature_extract = downloaded_gs_model.fit(train_data,
                                                     epochs=100, # fine-tune for a maximum of 100 epochs
                                                     steps_per_epoch=(len(train_data)),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),
                                                     callbacks=[create_tensorboard_callback(dir_name="training_logs",
                                                                                            experiment_name="efficientnetb0_101_classes_all_data_feature_extract"),
                                                                model_checkpoint_callback,
                                                                early_stopping,
                                                                reduce_lr])

In [ ]:
# Save model locally
downloaded_gs_model.save("drive/MyDrive/101_food_classes_100_percent_saved_big_dog_model_downloaded")

In [ ]:
# Evaluate mixed precision trained fine-tuned model (this should beat DeepFood's 77.4% top-1 accuracy)
results_fine_tuned_gs_model = downloaded_gs_model.evaluate(test_data)
results_fine_tuned_gs_model

In [ ]:
# Make predictions with model
preds_probs = downloaded_gs_model.predict(test_data, verbose=1)  # set verbosity to see how long is left